In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import auc
from sklearn import metrics

df = pd.read_csv('/Users/naresh/downloads/train.csv')
#df_train = df_train[0:46020]
#df_test = df_train[-30000:]
msk = np.random.rand(len(df)) < 0.7
df_train = df[msk]
df_test = df[~msk]
df_target = df_train['TARGET']
df_train = df_train.drop(['TARGET', 'ID'], axis=1)
df_test_target = df_test['TARGET']
df_test = df_test.drop(['TARGET','ID'], axis = 1)




probs = pd.DataFrame()
errors = pd.DataFrame()

X_train = df_train
y_train = df_target

X_train, X_test, y_train, y_test= train_test_split(X_train, y_train, test_size=0.3, random_state=0)
learning_rate = 0.01
n_estimators = 800
max_depth = 6
subsample = 0.9
colsample_bytree = 0.85
min_child_weight = 1  # default

eval_metrics = ['auc']
eval_sets = [(X_train, y_train), (X_test, y_test)]
xgb_model = XGBClassifier(seed=0, learning_rate=learning_rate, n_estimators=n_estimators,
                    min_child_weight=min_child_weight, max_depth=max_depth,
                    colsample_bytree=colsample_bytree, subsample=subsample,nthread = -1)
print("Fitting the model")
xgb = xgb_model.fit(X_train, y_train, eval_metric=eval_metrics, eval_set=eval_sets, verbose=False)
    
print("Predicting Probabilities")
probs['xgb'] = xgb.predict_proba(X_test)[:, -1]

print("Computing AUC")
auc_test = [xgb.evals_result_['validation_%d' % i]['auc'] for i in range(len(eval_sets))]
auc_test = np.array(auc_test, dtype=float).T

auc_best_round = np.argmax(auc_test, axis=0)
auc_best = [auc_test[auc_best_round[0], 0], auc_test[auc_best_round[1], 1]]

print('Best AUC train=%f (round=%d), test=%f (round=%d)' % (auc_best[0], auc_best_round[0], auc_best[1], auc_best_round[1]))
print('Validation')
test_probs = pd.DataFrame()
test_probs['xgb_valid'] = xgb.predict_proba(df_test)[:,-1]
print(test_probs['xgb_valid'].head())

fpr, tpr, thresholds = metrics.roc_curve(df_test_target, test_probs, pos_label=1)

a = float(auc(fpr,tpr))
print(a)

Fitting the model
Predicting Probabilities
Computing AUC
Best AUC train=0.906529 (round=799), test=0.837780 (round=600)
Validation
0    0.027952
1    0.004211
2    0.012208
3    0.014333
4    0.008440
Name: xgb_valid, dtype: float32
0.8352413580087394


In [ ]:
from sklearn.grid_search import GridSearchCV
from xgboost.sklearn import XGBClassifier

param_test1 = {
 'max_depth': [i for i in range(3,10,2)],
 'min_child_weight':[i for i in range(1,6,2)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [38]:
from sklearn.neural_network import MLPClassifier

X_train = StandardScaler().fit_transform(X_train)

nn = MLPClassifier(hidden_layer_sizes=(200,200,50), max_iter=100, alpha=1e-4,
                    algorithm='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init= 0.001)
X_test = StandardScaler().fit_transform(X_test)
df_test = StandardScaler().fit_transform(df_test)

nn.fit(X_train, y_train)
test_probs1 = pd.DataFrame()
test_probs1['mlp'] = nn.predict_proba(df_test1)[:, -1]
print(test_probs1['mlp'].head())
print()
fpr, tpr, thresholds = metrics.roc_curve(df_test_target, test_probs1, pos_label=1)


a = float(auc(fpr,tpr))
print(a)

Iteration 1, loss = 0.16352570
Iteration 2, loss = 0.14177073
Iteration 3, loss = 0.13801427
Iteration 4, loss = 0.13677800
Iteration 5, loss = 0.13627585
Iteration 6, loss = 0.13420606
Iteration 7, loss = 0.13236046
Iteration 8, loss = 0.13122264
Iteration 9, loss = 0.12953887
Iteration 10, loss = 0.13353598
Iteration 11, loss = 0.12928391
Iteration 12, loss = 0.12813579
Iteration 13, loss = 0.12660319
Iteration 14, loss = 0.12531548
Iteration 15, loss = 0.12946279
Iteration 16, loss = 0.12634359
Iteration 17, loss = 0.12359616
Iteration 18, loss = 0.12153390
Iteration 19, loss = 0.12203907
Iteration 20, loss = 0.12845549
Iteration 21, loss = 0.12023070
Iteration 22, loss = 0.11848059
Iteration 23, loss = 0.11748758
Iteration 24, loss = 0.11763244
Iteration 25, loss = 0.11491156
Iteration 26, loss = 0.11648367
Iteration 27, loss = 0.11425526
Iteration 28, loss = 0.11420380
Iteration 29, loss = 0.11053724
Iteration 30, loss = 0.11048128
Iteration 31, loss = 0.11776955
Iteration 32, los

In [39]:
from sklearn.neural_network import MLPClassifier

X_train = StandardScaler().fit_transform(X_train)

nn = MLPClassifier(hidden_layer_sizes=(200,50), max_iter=100, alpha=1e-4,
                    algorithm='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate= 'adaptive')
X_test = StandardScaler().fit_transform(X_test)
df_test = StandardScaler().fit_transform(df_test)

nn.fit(X_train, y_train)
test_probs1 = pd.DataFrame()
test_probs1['mlp'] = nn.predict_proba(df_test)[:, -1]
print(test_probs1['mlp'].head())
print()
fpr, tpr, thresholds = metrics.roc_curve(df_test_target, test_probs1, pos_label=1)


a = float(auc(fpr,tpr))
print(a)

Iteration 1, loss = 0.15999335
Iteration 2, loss = 0.14222169
Iteration 3, loss = 0.13778863
Iteration 4, loss = 0.13639491
Iteration 5, loss = 0.13546905
Iteration 6, loss = 0.13368440
Iteration 7, loss = 0.13218417
Iteration 8, loss = 0.13058078
Iteration 9, loss = 0.12937945
Iteration 10, loss = 0.13287769
Iteration 11, loss = 0.12860296
Iteration 12, loss = 0.12750545
Iteration 13, loss = 0.12638896
Iteration 14, loss = 0.12586441
Iteration 15, loss = 0.12917647
Iteration 16, loss = 0.12461050
Iteration 17, loss = 0.12260380
Iteration 18, loss = 0.12221281
Iteration 19, loss = 0.12201874
Iteration 20, loss = 0.12959499
Iteration 21, loss = 0.12285322
Iteration 22, loss = 0.12027138
Iteration 23, loss = 0.11900718
Iteration 24, loss = 0.11889244
Iteration 25, loss = 0.11775066
Iteration 26, loss = 0.11767397
Iteration 27, loss = 0.11603549
Iteration 28, loss = 0.11543907
Iteration 29, loss = 0.11398784
Iteration 30, loss = 0.11326235
Iteration 31, loss = 0.11928200
Iteration 32, los